In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [3]:
class Strategy_comp_mul(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('p_stake', 100),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('rsi_short', 6),
        ('rsi_long', 12),
        ('wave_period', 7),
        ('wr', 14),

        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Record ATR Initial
        self.atr_initial = self.broker.get_cash()
        self.inds = dict()
        self.profit_i = dict()
        self.data_tracker = None

        for d in self.datas:

            self.inds[d] = dict()
            self.profit_i[d._name] = 0

            # Keep references to lines in the data[0] dataseries
            self.inds[d]['dataclose'] = d.close
            self.inds[d]['datahigh'] = d.high
            self.inds[d]['datalow'] = d.low

            # Keep references to executed order & price
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None
                
            # Add indicators
            ## Wave indicator
            ### MovingAverageSimple indicator
            self.inds[d]['sma'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma'](0) > self.inds[d]['sma'](-1), 
                                              self.inds[d]['sma'](-1) < self.inds[d]['sma'](-2), 
                                              self.inds[d]['sma'](-2) < self.inds[d]['sma'](-3))

            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma'](0) < self.inds[d]['sma'](-1), 
                                               self.inds[d]['sma'](-1) > self.inds[d]['sma'](-2), 
                                               self.inds[d]['sma'](-2) > self.inds[d]['sma'](-3))

            ## MACD indicator
            self.inds[d]['macdhisto'] = bt.indicators.MACDHisto(d)
            self.inds[d]['macd_buy'] = bt.And(self.inds[d]['macdhisto'](0) > 0, self.inds[d]['macdhisto'](-1) < 0)
            self.inds[d]['macd_sell'] = bt.And(self.inds[d]['macdhisto'](0) < 0, self.inds[d]['macdhisto'](-1) > 0)

            ## Bollinger Bands indicator
            self.inds[d]['bolling'] = bt.indicators.BollingerBands(d, 
                                                        period=self.params.bollinger_per, 
                                                        devfactor=self.params.bollinger_dev)
            self.inds[d]['bolling_buy'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].top(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].top(-1)), 
                                                bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].bot(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].bot(-1)))
            self.inds[d]['bolling_sell'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].top(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].top(-1)), 
                                                 bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].bot(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].bot(-1)))

            ## RSI indicator
            self.inds[d]['rsi_s'] = bt.indicators.RelativeStrengthIndex(d, period=self.params.rsi_short, safediv=True)
            self.inds[d]['rsi_l'] = bt.indicators.RelativeStrengthIndex(d, period=self.params.rsi_long, safediv=True)
            self.inds[d]['rsi_buy'] = bt.And(self.inds[d]['rsi_s'](0) > self.inds[d]['rsi_l'](0), 
                                             self.inds[d]['rsi_s'](-1) < self.inds[d]['rsi_l'](-1))
            self.inds[d]['rsi_sell'] = bt.And(self.inds[d]['rsi_s'](0) < self.inds[d]['rsi_l'](0), 
                                              self.inds[d]['rsi_s'](-1) > self.inds[d]['rsi_l'](-1))

            ## WilliamR indicator
            self.inds[d]['wr'] = bt.indicators.WilliamsR(d, period=self.params.wr)
            self.inds[d]['wr_buy'] = self.inds[d]['wr'] < -80
            self.inds[d]['wr_sell'] = self.inds[d]['wr'] > -20
        
            ## ATR indicator
            self.inds[d]['tr'] = bt.indicators.Max((self.inds[d]['datahigh'](0) - self.inds[d]['datalow'](0)), 
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datahigh'](0)),
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datalow'](0)))
            self.inds[d]['atr'] = bt.indicators.SimpleMovingAverage(self.inds[d]['tr'], period=self.params.atr_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.inds[self.data_tracker]['wave_buy']:
                    self.inds[self.data_tracker]['buyprice']
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'],
                          order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

            else:  # Sell
                self.inds[self.data_tracker]['sellprice'] = order.executed.price
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                        (order.info['name'],
                         order.executed.price,
                         -order.executed.size,
                         self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                        (order.info['name'],
                         self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益 代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
    

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
        # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return
            buy_indicator = (self.params.buy_signal == 'b' and self.inds[d]['bolling_buy']) or \
                            (self.params.buy_signal == 'm' and self.inds[d]['macd_buy']) or \
                            (self.params.buy_signal == 'w' and self.inds[d]['wave_buy'] and self.inds[d]['dataclose'][0] > self.inds[d]['buyprice']) or \
                            (self.params.buy_signal == 'r' and self.inds[d]['rsi_buy']) or \
                            (self.params.buy_signal == 'wr' and self.inds[d]['wr_buy'])


            sell_indicator = (self.params.sell_signal == 'b' and self.inds[d]['bolling_sell']) or \
                            (self.params.sell_signal == 'm' and self.inds[d]['macd_sell']) or \
                            (self.params.sell_signal == 'w' and self.inds[d]['wave_sell']) or \
                            (self.params.sell_signal == 'r' and self.inds[d]['rsi_sell']) or \
                            (self.params.sell_signal == 'wr' and self.inds[d]['wr_sell'])

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))
                #self.log('Sell CREATE(Close), Code: %s, Price: %.2f, Current Position: %i' % 
                #         (d._name, 
                #          self.inds[d]['dataclose'][0],
                #          self.getposition(d).size))
                #self.close(data=d, name=d._name)

            elif buy_indicator and self.inds[d]['dataclose'][0] > self.inds[d]['bolling'].mid[0]:
                if self.params.using_atr:
                    opt_pos = np.floor(self.atr_initial * self.params.atr_percent / self.params.p_stake / self.inds[d]['atr'][0])
                    if opt_pos*self.params.p_stake > 0 and self.getposition(d).size < opt_pos*self.params.p_stake:
                        buy_list.append((d, 
                                        opt_pos*self.params.p_stake-self.getposition(d).size, 
                                        (opt_pos*self.params.p_stake-self.getposition(d).size)*self.inds[d]['dataclose'][0]))
                        #self.log('BUY CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                        #        (d._name, 
                        #         self.inds[d]['dataclose'][0], 
                        #         opt_pos*self.params.p_stake-self.getposition(d).size,
                        #         self.getposition(d).size))
                        #self.buy(data=d, size=opt_pos*self.params.p_stake-self.getposition(d).size, name=d._name)

                    #elif opt_pos*self.params.p_stake > 0 and self.getposition(d).size > opt_pos*self.params.p_stake:
                    #    sell_list.append((d, 
                    #    self.getposition(d).size-opt_pos*self.params.p_stake))
                        #self.log('SELL CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                        #        (d._name, 
                        #         self.inds[d]['dataclose'][0], 
                        #         self.getposition(d).size-opt_pos*self.params.p_stake,
                        #         self.getposition(d).size))
                        #self.sell(data=d, size=self.getposition(d).size-opt_pos*self.params.p_stake, name=d._name)
                else:
                    buy_list.append((d, 
                                     opt_pos*self.params.p_stake, 
                                     opt_pos*self.params.p_stake*self.inds[d]['dataclose'][0]))
                    #self.log('BUY CREATE, Code: %s, Price: %.2f, Lots: %i, Current Position: %i' % 
                    #        (d._name, 
                    #         self.inds[d]['dataclose'][0],
                    #         self.params.p_stake,
                    #         self.getposition(d).size))
                    #self.buy(data=d, size=self.params.p_stake, name=d._name)
    
        for s_order in sell_list:
            self.data_tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i, 现有持仓: %i' % 
                        (s_order[0]._name, 
                        self.inds[s_order[0]]['dataclose'][0],
                        s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size,
                        self.getposition(s_order[0]).size))
            self.sell(data=s_order[0], 
                        size=s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size, 
                        name=s_order[0]._name)
        
        #buy_list = sorted(buy_list, key=lambda x: x[2], reverse=False)
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.data_tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i, 现有持仓: %i' % 
                        (b_order[0]._name, 
                        self.inds[b_order[0]]['dataclose'][0], 
                        b_order[1],
                        self.getposition(b_order[0]).size))
            self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)


    def stop(self):
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * d.close[0]
        #self.log('Ending Value:%.2f.' %
        #        (self.cerebro.broker.getvalue()), doprint=False)
        #print(sorted(self.profit_i.items(), key=lambda x: x[1]))
        #print(sum([x[1] for x in self.profit_i.items()]))
        return

# 全市场回测

### 数据下载

In [8]:
s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365 * 2)
e_date = datetime.date(2020, 12, 31)

#s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
#e_date = datetime.datetime.now()

stock_index = '399300.SZ'

# Download Data
utilsJ.index_to_csv_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',
                            stock_index, 0.2, s_date, e_date, fpath='.\\Data\\2019\\')

### 最优策略回测

In [7]:
profit_dict = {}

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

def getdata_mul(s_code, start=datetime.date(2020, 12, 31)-datetime.timedelta(days = 385),
                end = datetime.date(2020, 12, 31)):
    global cerebro
    df = pd.read_csv('.\\Data\\2019\\'+s_code+'.csv', 
                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)
    cerebro.adddata(data, name=s_code)
    return

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)



#stock_index = '399300.SZ'
#pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
#index_list = np.unique(pro.index_weight(index_code=stock_index,
#                                        start_date=s_date.strftime('%Y%m%d'),
#                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()


if __name__ == '__main__':
    buy_s = ['b','m','w', 'r', 'wr']
    sell_s = ['b','m','w', 'r', 'wr']
    for s in product(*[buy_s, sell_s]):
        random.seed(10086)
        for i in range(3):
            # Create a cerebro entity
            cerebro = bt.Cerebro(tradehistory=True)

            # Add a strategy
            strats = cerebro.addstrategy(Strategy_comp_mul, printlog=False, using_atr=True, atr_percent=0.01, buy_signal = s[0], sell_signal = s[1])

            t_list = [utilsJ.MyThread(getdata_mul, (s_code, s_date, e_date)) for s_code in index_list]
            [t.start() for t in t_list]
            [t.join() for t in t_list]
            #for s_code in index_list:
            #    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv', 
            #                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
            #    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)

                # Add the index Data Feed to Cerebo
            #    cerebro.adddata(data, name=s_code)
            
            # Set cash inside the strategy
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(1000000)

            # Set commission
            #cerebro.broker.setcommission()

            # Print out the starting conditions
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Run over everything
            cerebro.run(maxcpus=8)

            # Print out the final result
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
            if (s[0], s[1]) not in profit_dict.keys():
                profit_dict[(s[0], s[1])] = []
            profit_dict[(s[0], s[1])].append((final_value - start_value) / start_value * 100)

(b, b) Net Profit: 1.16%
(b, b) Net Profit: -2.74%
(b, b) Net Profit: 11.16%
(b, m) Net Profit: 18.57%
(b, m) Net Profit: 24.88%
(b, m) Net Profit: 8.52%
(b, w) Net Profit: 17.84%
(b, w) Net Profit: 18.75%
(b, w) Net Profit: -10.59%
(b, r) Net Profit: 22.93%
(b, r) Net Profit: 4.98%
(b, r) Net Profit: 1.35%
(b, wr) Net Profit: 12.47%
(b, wr) Net Profit: -5.15%
(b, wr) Net Profit: 2.39%
(m, b) Net Profit: 33.35%
(m, b) Net Profit: 8.01%
(m, b) Net Profit: 2.43%
(m, m) Net Profit: -1.35%
(m, m) Net Profit: -9.37%
(m, m) Net Profit: -8.78%
(m, w) Net Profit: 17.63%
(m, w) Net Profit: 3.10%
(m, w) Net Profit: -10.22%
(m, r) Net Profit: -0.92%
(m, r) Net Profit: 2.45%
(m, r) Net Profit: 17.22%
(m, wr) Net Profit: 17.86%
(m, wr) Net Profit: -9.89%
(m, wr) Net Profit: 3.03%
(w, b) Net Profit: 8.30%
(w, b) Net Profit: 7.92%
(w, b) Net Profit: 37.50%
(w, m) Net Profit: 10.31%
(w, m) Net Profit: 6.94%
(w, m) Net Profit: 23.06%
(w, w) Net Profit: 17.00%
(w, w) Net Profit: 28.59%
(w, w) Net Profit

### 循环回测

In [6]:
profit_dict = {}

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

def getdata_mul(s_code, start=datetime.date(2020, 12, 31)-datetime.timedelta(days = 385),
                end = datetime.date(2020, 12, 31)):
    global cerebro
    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv', 
                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)
    cerebro.adddata(data, name=s_code)
    return

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)



stock_index = '399300.SZ'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()


if __name__ == '__main__':
    for s in []:
        for i in range(3):
            random.seed(10086)
            # Create a cerebro entity
            cerebro = bt.Cerebro(tradehistory=True)

            # Add a strategy
            strats = cerebro.addstrategy(Strategy_comp_mul, printlog=False, using_atr=True, atr_percent=0.01, buy_signal = s[0], sell_signal = s[1])

            t_list = [utilsJ.MyThread(getdata_mul, (s_code, s_date, e_date)) for s_code in index_list]
            [t.start() for t in t_list]
            [t.join() for t in t_list]
            #for s_code in index_list:
            #    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv', 
            #                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
            #    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)

                # Add the index Data Feed to Cerebo
            #    cerebro.adddata(data, name=s_code)
            
            # Set cash inside the strategy
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(1000000)

            # Set commission
            #cerebro.broker.setcommission()

            # Print out the starting conditions
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Run over everything
            cerebro.run(maxcpus=8)

            # Print out the final result
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
            print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
            if (s[0], s[1]) not in profit_dict.keys():
                profit_dict[(s[0], s[1])] = []
            profit_dict[(s[0], s[1])].append((final_value - start_value) / start_value * 100)

KeyboardInterrupt: 

In [117]:
random.seed(10086)

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

stock_index = '399300.SZ'
#pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
#index_list = np.unique(pro.index_weight(index_code=stock_index,
#                                        start_date=s_date.strftime('%Y%m%d'),
#                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
profit = []
cerebro = bt.Cerebro(tradehistory=True)

# Add a strategy
strats = cerebro.addstrategy(Strategy_comp_mul, printlog=True, using_atr=True, atr_percent=0.00008, buy_signal = 'wr', sell_signal = 'm') 

for s_code in index_list:
    df = pd.read_csv('.\\Data\\2020\\' + s_code + '.csv')
    df.index=pd.to_datetime(df.trade_date)
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data, name = s_code)

# Set cash inside the strategy
cerebro.broker = bt.brokers.BackBroker(coc=True)
cerebro.broker.setcash(1000000)

# Set commission
#cerebro.broker.setcommission()

# Print out the starting conditions
start_value = cerebro.broker.getvalue()
#print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run(maxcpus=4)

# Print out the final result
final_value = cerebro.broker.getvalue()
#print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
#profit.append((final_value - start_value) / start_value * 100)

2020-08-13: 买单创建, 代码: 002120.SZ, 价格: 23.28, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 603993.SH, 价格: 4.31, 股数: 300, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601298.SH, 价格: 6.27, 股数: 400, 现有持仓: 0
2020-08-13: 买单创建, 代码: 600998.SH, 价格: 17.68, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601899.SH, 价格: 5.72, 股数: 200, 现有持仓: 0
2020-08-13: 买单创建, 代码: 002558.SZ, 价格: 20.33, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601212.SH, 价格: 3.11, 股数: 300, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601555.SH, 价格: 10.23, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601328.SH, 价格: 4.76, 股数: 1500, 现有持仓: 0
2020-08-13: 买单创建, 代码: 002602.SZ, 价格: 10.59, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601600.SH, 价格: 3.22, 股数: 600, 现有持仓: 0
2020-08-13: 买单创建, 代码: 600208.SH, 价格: 3.33, 股数: 700, 现有持仓: 0
2020-08-13: 买单创建, 代码: 002252.SZ, 价格: 9.40, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 600109.SH, 价格: 13.87, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601225.SH, 价格: 7.72, 股数: 400, 现有持仓: 0
2020-08-13: 买单创建, 代码: 600061.SH, 价格: 14.54, 股数: 100, 现有持仓: 0
2020-08-13: 买单创建, 代码: 601988.SH,